In [1]:
!pip install ortools

In [2]:
from ortools.sat.python import cp_model

In [11]:
#crear un modelo CSP
model = cp_model.CpModel()

In [12]:
#Datos
n_casas = 5

nacion = ["Ingles", "Español", "Japones", "Italiano", "Noruego"]
profesion = ["Pintor", "Escultor", "Diplomatico", "Medico", "Violinista"]
animal = ["Zorro", "Caballo", "Cebra", "Perro", "Caracol"]
bebida = ["Te", "Cafe", "Agua", "Jugo", "Leche"]
color = ["Rojo", "Amarillo", "Verde", "Blanco", "Azul"]


In [13]:
#Variables y Dominio

N = [model.NewIntVar(0, n_casas-1, f"{x}") for x in nacion]
P = [model.NewIntVar(0, n_casas-1, f"{x}") for x in profesion]
A = [model.NewIntVar(0, n_casas-1, f"{x}") for x in animal]
B = [model.NewIntVar(0, n_casas-1, f"{x}") for x in bebida]
C = [model.NewIntVar(0, n_casas-1, f"{x}") for x in color]
print(N)
print(P)
print(A)
print(B)
print(C)

[Ingles(0..4), Español(0..4), Japones(0..4), Italiano(0..4), Noruego(0..4)]
[Pintor(0..4), Escultor(0..4), Diplomatico(0..4), Medico(0..4), Violinista(0..4)]
[Zorro(0..4), Caballo(0..4), Cebra(0..4), Perro(0..4), Caracol(0..4)]
[Te(0..4), Cafe(0..4), Agua(0..4), Jugo(0..4), Leche(0..4)]
[Rojo(0..4), Amarillo(0..4), Verde(0..4), Blanco(0..4), Azul(0..4)]


In [39]:
#Restricciones

model.AddAllDifferent(N)
model.AddAllDifferent(P)
model.AddAllDifferent(A)
model.AddAllDifferent(B)
model.AddAllDifferent(C)


model.Add(N[nacion.index("Ingles")] == C[color.index("Rojo")])
model.Add(N[nacion.index("Español")] == A[animal.index("Perro")])
model.Add(N[nacion.index("Japones")] == P[profesion.index("Pintor")])
model.Add(N[nacion.index("Italiano")] == B[bebida.index("Te")])
model.Add(N[nacion.index("Noruego")] == 0)
model.Add(C[color.index("Verde")] == B[bebida.index("Cafe")])
model.Add(C[color.index("Blanco")] == C[color.index("Verde")]-1)
model.Add(P[profesion.index("Escultor")] == A[animal.index("Caracol")])
model.Add(P[profesion.index("Diplomatico")] == C[color.index("Amarillo")])
model.Add(B[bebida.index("Leche")] == 2)
model.Add(N[nacion.index("Noruego")] == C[color.index("Azul")]-1)
model.Add(P[profesion.index("Violinista")] == B[bebida.index("Jugo")])

model.AddAbsEquality( 1, (A[animal.index("Zorro")] - P[profesion.index("Medico")]) )
model.AddAbsEquality( 1, (A[animal.index("Caballo")] - P[profesion.index("Diplomatico")]) )

#model.Add(x != y)
#model.Add(sum([G[i][f] if(gender[i]==0) else 0 for i in range(n_crew)]) >= min_hostesess[f])

#model.AddMultiplicationEquality(c, [a,b]) #c == a * b

#model.AddAllDifferent([s,e,n,d,m,o,r,y])
#model.AddAtLeastOne([G[i][f] if (i in french) else 0 for i in range(n_crew)])

#model.AddAllowedAssignments((seq[4], seq[9]), rhymes_table) # First rhyme
#model.AddForbiddenAssignments( # also named table constraint, or extensional constraint
#                                (seq[i], seq[i + 1], seq[i + 2], seq[i + 3]),  # variables
#                                plagiarism_table)     # table

#model.Add(x >= 5).OnlyEnforceIf(b) #b => (x >= 5)
#model.Add(x < 5).OnlyEnforceIf(b.Not()) #not(b) => not(x >= 5) --> (x >= 5) => b

In [ ]:
#Optimizacion

#Función Objetivo

#model.Maximize(a+b+c)
#model.Minimize(numero_colores)


#heurísticas
#model.AddDecisionStrategy(variables, heurística de variable, heurística de valor)
#en este caso heurística de variable=elegir al primero en orden [a,b,c]
#en este caso heurística de valor=elegir el máximo valor posible

#model.AddDecisionStrategy([a,b,c], cp_model.CHOOSE_FIRST, cp_model.SELECT_MAX_VALUE)

In [40]:
#Solver Normal
solver = cp_model.CpSolver() 
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  for i in range(n_casas):
      result = "Casa " + str(i) + ": "
      for j in range(n_casas):
        if(solver.Value(N[j]) == i):
          result += str(N[j]) + ", "

        if(solver.Value(P[j]) == i):
          result += str(P[j]) + ", "
          
        if(solver.Value(A[j]) == i):
          result += str(A[j]) + ", "
          
        if(solver.Value(B[j]) == i):
          result += str(B[j]) + ", "
          
        if(solver.Value(C[j]) == i):
          result += str(C[j]) + ", "

      print(result[:-2])

  

#Tiempo
print(solver.WallTime())

Casa 0: Zorro, Amarillo, Diplomatico, Agua, Noruego
Casa 1: Te, Caballo, Italiano, Medico, Azul
Casa 2: Ingles, Rojo, Escultor, Caracol, Leche
Casa 3: Español, Perro, Jugo, Blanco, Violinista
Casa 4: Pintor, Cafe, Japones, Cebra, Verde
0.00831497


In [41]:
#Solver All Solutions
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [44]:
#Solver All Solutions
solver = cp_model.CpSolver()
Res = N+P+A+B+C
solution_printer = VarArraySolutionPrinter(Res)
status = solver.SearchForAllSolutions(model, solution_printer)

print('# solutions:', solution_printer.solution_count())
print(solver.WallTime())
# Enumerate all solutions.
#solver.parameters.enumerate_all_solutions = True
#status = solver.Solve(model, solution_printer)




Ingles=2 Español=3 Japones=4 Italiano=1 Noruego=0 Pintor=4 Escultor=2 Diplomatico=0 Medico=1 Violinista=3 Zorro=0 Caballo=1 Cebra=4 Perro=3 Caracol=2 Te=1 Cafe=4 Agua=0 Jugo=3 Leche=2 Rojo=2 Amarillo=0 Verde=4 Blanco=3 Azul=1 
# solutions: 1
0.011913493
